In [20]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
#jupyter内で画像を表示
%matplotlib inline 
#別ウィンドウで画像表示
# %matplotlib qt 
import csv
import json
from ipywidgets import interact, interactive, fixed, RadioButtons
import ipywidgets as widgets
from IPython.display import display
import pathlib
seed = '10009'
node_num = '400'
shadow = 'shadow10_2'
sigo_path = pathlib.Path('../../get_data/recover_test2/' + shadow + '/sigo/sigo-seed_' +seed +'nodenum_' +node_num+'.csv')
lsgo_path = pathlib.Path('../../get_data/recover_test2/' + shadow + '/lsgo/lsgo-seed_' +seed +'nodenum_' +node_num+'.csv')
jbr_path = pathlib.Path('../../get_data/recover_test2/' + shadow + '/jbr/sigo-seed_' +seed +'nodenum_' +node_num+'.csv')

# sigo_path = pathlib.Path('../../ns3.30/data/no_buildings/sigo/sigo-seed_' +seed +'nodenum_' +node_num+'.csv')
# lsgo_path = pathlib.Path('../../ns3.30/data/no_buildings/lsgo/lsgo-seed_' +seed +'nodenum_' +node_num+'.csv')

# csvファイルの取得
pandasを利用：https://note.nkmk.me/python-pandas-value-counts/

In [21]:
sigo_df = pd.read_csv(sigo_path)
lsgo_df = pd.read_csv(lsgo_path)
jbr_df = pd.read_csv(jbr_path)

In [22]:
sigo_df

,source_x,source_y,recv_x,recv_y,time,recv_priority,hopcount,recv_id,source_id,destination_id,destination_x,destination_y,pri_1,pri_2,pri_3,pri_4,pri_5
0,1,212,1,247,10300200,5,1,176,379,257,580,801,32,56,348,174,176
1,1,212,1,282,10300200,1,1,32,379,257,580,801,32,56,348,174,176
2,1,212,-1,355,10300200,4,1,174,379,257,580,801,32,56,348,174,176
3,1,212,-1,387,10300200,3,1,348,379,257,580,801,32,56,348,174,176
4,1,282,-1,319,10300564,3,2,64,32,257,580,801,348,282,64,174,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,801,723,798,800,19323480,2,10,74,261,301,868,798,94,74,131,6,10000000
502,802,793,818,801,19323844,5,11,51,94,301,868,798,252,301,196,329,51
503,802,793,840,801,19323844,3,11,196,94,301,868,798,252,301,196,329,51
504,802,793,876,801,19323844,1,11,252,94,301,868,798,252,301,196,329,51


In [23]:
jbr_df

,source_x,source_y,recv_x,recv_y,time,recv_priority,hopcount,recv_id,source_id,destination_id,destination_x,destination_y,pri_1,pri_2,pri_3,pri_4,pri_5,pri_recover
0,1,212,1,247,10300200,5,1,176,379,257,580,801,32,56,348,174,176,0
1,1,212,1,282,10300200,1,1,32,379,257,580,801,32,56,348,174,176,0
2,1,212,-1,355,10300200,4,1,174,379,257,580,801,32,56,348,174,176,0
3,1,212,-1,387,10300200,3,1,348,379,257,580,801,32,56,348,174,176,0
4,1,282,-1,319,10300564,3,2,64,32,257,580,801,348,282,64,174,43,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,801,648,801,692,19327480,5,10,334,371,301,868,798,74,94,131,261,334,0
495,801,648,801,723,19327480,4,10,261,371,301,868,798,74,94,131,261,334,0
496,801,648,801,748,19327480,3,10,131,371,301,868,798,74,94,131,261,334,0
497,801,648,802,793,19327480,2,10,94,371,301,868,798,74,94,131,261,334,0


In [24]:
print(sigo_path)
print(lsgo_path)
print(jbr_path)

..\..\get_data\recover_test2\shadow10_2\sigo\sigo-seed_10009nodenum_400.csv
..\..\get_data\recover_test2\shadow10_2\lsgo\lsgo-seed_10009nodenum_400.csv
..\..\get_data\recover_test2\shadow10_2\jbr\sigo-seed_10009nodenum_400.csv


# recovery packet の多さ

In [25]:
jbr_df['pri_recover'].value_counts().to_dict()

{0: 495, 1: 4}

# destination_idごとのパケット数

In [26]:
lsgo_df['destination_id'].value_counts().to_dict()

{148: 112,
 257: 99,
 301: 90,
 6: 50,
 69: 48,
 377: 37,
 133: 34,
 16: 26,
 273: 15,
 128: 14}


destination_id を重複なしにリスト保存

In [27]:
sigo_df['destination_id'].value_counts().to_dict()

{69: 95,
 301: 84,
 257: 61,
 133: 55,
 6: 47,
 128: 41,
 377: 37,
 16: 37,
 148: 31,
 273: 18}

In [28]:
jbr_df['destination_id'].value_counts().to_dict()

{69: 95,
 128: 77,
 301: 62,
 257: 61,
 133: 55,
 6: 47,
 148: 31,
 377: 27,
 16: 26,
 273: 18}

In [29]:
dest_list =lsgo_df['destination_id'].value_counts().index.tolist()
dest_list.sort()
len(dest_list)

10

In [30]:
dest_list = sigo_df['destination_id'].value_counts().index.tolist()
dest_list.sort()
len(dest_list)

10

In [31]:
dest_list = jbr_df['destination_id'].value_counts().index.tolist()
dest_list.sort()
len(dest_list)

10

sigoとlsgoそれぞれ目的地に届いたidだけをlistに保存

In [32]:
sigo_reach_list = sigo_df[sigo_df['recv_id'] == sigo_df['destination_id']]['destination_id'].value_counts().index.tolist()
lsgo_reach_list = lsgo_df[lsgo_df['recv_id'] == lsgo_df['destination_id']]['destination_id'].value_counts().index.tolist()
jbr_reach_list = jbr_df[jbr_df['recv_id'] == jbr_df['destination_id']]['destination_id'].value_counts().index.tolist()
sigo_reach_list.sort()
lsgo_reach_list.sort()
jbr_reach_list.sort()

print('sigo:',sigo_reach_list)
print('lsgo:',lsgo_reach_list)
print('jbr:',jbr_reach_list)

sigo: [6, 16, 69, 128, 133, 148, 257, 273, 301, 377]
lsgo: [6, 16, 69, 148, 257, 301, 377]
jbr: [6, 16, 69, 133, 148, 257, 273, 377]


# 平均delayを返すメソッド

In [33]:
def packet_delay(path, spefic=0): #spefic 0 or id   0の時平均を返す idのときそのdestination id のdelayを返す
    delay_list = []
    #csvファイルの取得
    path_df = pd.read_csv(path)
    #目的地に届いたidだけをlistに保存
    reach_list = path_df[path_df['recv_id'] == path_df['destination_id']]['destination_id'].value_counts().index.tolist()
    if(spefic == 0):
        #reach_listを回す
        for reach_id in reach_list:
            source_time = path_df[path_df['destination_id'] == reach_id]['time']
            recv_time = path_df[(path_df['destination_id'] == reach_id) & (path_df['destination_id'] == path_df['recv_id'])]['time']
            delay = int(recv_time.head(1).iloc[-1]) - int(source_time.head(1).iloc[-1])
            delay = delay / 100000
        #         print('desID:', reach_id, 'delay:',delay)
            delay_list.append(delay)
        delay_mean = np.mean(delay_list)
        return delay_mean
    else:
        source_time = path_df[path_df['destination_id'] == spefic]['time']
        recv_time = path_df[(path_df['destination_id'] == spefic) & (path_df['destination_id'] == path_df['recv_id'])]['time']
        delay = int(recv_time.head(1).iloc[-1]) - int(source_time.head(1).iloc[-1])
        delay = delay / 100000
        return delay

In [34]:
packet_delay(sigo_path)

0.261522

In [35]:
packet_delay(lsgo_path)

0.29981

In [36]:
packet_delay(jbr_path)

0.3223375

# 軌跡描画

In [37]:
def draw_source_area(log, img_area):
    cv2.rectangle(img_area, (log.source_x - 15, log.source_y - 15), (log.source_x + 15, log.source_y + 15), (255, 255, 0),thickness=8)
    cv2.rectangle(img_area, (log.destination_x - 15, log.destination_y - 15), (log.destination_x + 15, log.destination_y + 15), (255, 255, 0),thickness=8)
    #source nodeを視覚で囲む
    cv2.rectangle(img_area, (log.source_x - 15, log.source_y - 15), (log.source_x + 15, log.source_y + 15), (255, 255, 0),thickness=8)

    #方眼
    plt.figure(figsize=(10, 10))
    print('グラフサイズ')
    plt.grid(color='w', linestyle='dotted', linewidth=1.0)
    plt.xlim([0, 1000])
    plt.xticks([1000,800,600,400,200, 0]) 
    plt.ylim([0, 1000])
    plt.yticks([1000,800,600,400,200, 0])  #メモリの間隔を指定 
    print('source_area 描画', log.source_x)

# パケットの軌跡を描画
def draw(log,count,img_area,protocol,text):
    recover_line_color = (255, 0, 0)
    if protocol == 'sigo':
        line_color = (0, 255, 0)
        thick = 7
        des_thick = 8
    elif protocol == 'lsgo':
        line_color = (255, 0, 0)
        thick = 2
        des_thick = 2
    elif protocol == 'jbr':
        line_color = (0, 255, 255)
        thick = 6
        des_thick = 2
    else:
        line_color = (255, 0, 0)
        thick = 2
        des_thick = 2
        
    if(protocol != 'sigo' and protocol != 'lsgo'):  #recovery routing だった場合
        if(log.pri_recover == 1):
            cv2.arrowedLine(img_area,(log.source_x, log.source_y), 
                                (log.recv_x, log.recv_y), recover_line_color, thickness=thick, tipLength=0.3)
            cv2.circle(img=img_area, center=(log.source_x,
                                   log.source_y), radius=5, color=(255,0,0), thickness=-1)
            cv2.circle(img=img_area, center=(log.recv_x,
                                   log.recv_y), radius=5, color=(255,0,0), thickness=-1)
        else:
            cv2.arrowedLine(img_area,(log.source_x, log.source_y), 
                                (log.recv_x, log.recv_y), line_color, thickness=thick, tipLength=0.1)
            cv2.circle(img=img_area, center=(log.source_x,
                                   log.source_y), radius=5, color=(255,0,0), thickness=-1)
            cv2.circle(img=img_area, center=(log.recv_x,
                                   log.recv_y), radius=5, color=(255,0,0), thickness=-1)
    
    else: # normal routing sigo or lsgo
        cv2.arrowedLine(img_area,(log.source_x, log.source_y), 
                            (log.recv_x, log.recv_y), line_color, thickness=thick, tipLength=0.1)
        cv2.circle(img=img_area, center=(log.source_x,
                               log.source_y), radius=5, color=(255,0,0), thickness=-1)
        cv2.circle(img=img_area, center=(log.recv_x,
                               log.recv_y), radius=5, color=(255,0,0), thickness=-1)
        
    
    
    #受け取ったのが宛先だったならば
    if log.recv_id == log.destination_id:
        cv2.drawMarker(img_area, (log.recv_x, log.recv_y), line_color, thickness=des_thick, markerType=cv2.MARKER_TILTED_CROSS, markerSize=25)
    if text == 'あり':    
        cv2.putText(img_area, str(log.source_id), (log.source_x + 10,log.source_y +10 ), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), thickness=2)
        cv2.putText(img_area, str(log.recv_id), (log.recv_x + 10,log.recv_y +10 ), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), thickness=2)

In [38]:
img_area = np.zeros(shape=(1000,1000, 3), dtype=np.int16)
protocol_list = ["both","lsgo", "sigo", "jbr"]
text_list = ["なし", "あり"]
print('sigo:',sigo_reach_list,'PDR:',len(sigo_reach_list)/len(sigo_df['destination_id'].value_counts().index.tolist()))
print('lsgo:',lsgo_reach_list,'PDR:',len(lsgo_reach_list)/len(lsgo_df['destination_id'].value_counts().index.tolist()))
print('jbr:',jbr_reach_list,'PDR:',len(jbr_reach_list)/len(jbr_df['destination_id'].value_counts().index.tolist()))
print('sigo delay:', packet_delay(sigo_path))
print('lsgo delay:', packet_delay(lsgo_path))
print('jbr delay:', packet_delay(jbr_path))

#packetの軌跡を描画 
@interact(minHop=(0,19,1), maxHop=(1,21, 1), desId=dest_list, protocol = protocol_list, text = text_list)
def packetDraw(protocol, desId, minHop=0, maxHop = 21, text = "なし"):
    
    img_area = np.zeros(shape=(2100,2100, 3), dtype=np.int16)
    count = 1
    if protocol == 'sigo':     
        for log in sigo_df[sigo_df['destination_id'] == desId].itertuples():
            if count == 1:
                draw_source_area(log, img_area)
                count = count +1
            if log.hopcount >= minHop and log.hopcount <= maxHop:
                draw(log,count,img_area,protocol,text)
                
        plt.imshow(img_area)
    elif protocol == 'lsgo':
        
        for log in lsgo_df[lsgo_df['destination_id'] == desId].itertuples():
            if count == 1:
                draw_source_area(log, img_area)
                count = count +1
                
            if log.hopcount >= minHop and log.hopcount <= maxHop:
                draw(log,count,img_area,protocol,text)
 
        plt.imshow(img_area)
    
    elif protocol == 'jbr':
        
        for log in jbr_df[jbr_df['destination_id'] == desId].itertuples():
            if count == 1:
                draw_source_area(log, img_area)
                count = count +1
                
            if log.hopcount >= minHop and log.hopcount <= maxHop:
                draw(log,count,img_area,protocol,text)
 
        plt.imshow(img_area)
    
    else:
        if(desId in sigo_reach_list):
            print('sigo packet delay id', desId, 'delay', packet_delay(sigo_path, desId))
        if(desId in lsgo_reach_list):
            print('lsgo packet delay id', desId, 'delay', packet_delay(lsgo_path, desId))
        if(desId in jbr_reach_list):
            print('jbr packet delay id', desId, 'delay', packet_delay(jbr_path, desId))
        
        for log in sigo_df[sigo_df['destination_id'] == desId].itertuples():
            if count == 1:
                draw_source_area(log, img_area)
                count = count +1
                
            if log.hopcount >= minHop and log.hopcount <= maxHop:
                draw(log,count,img_area,'sigo',text)
                
        for log in lsgo_df[lsgo_df['destination_id'] == desId].itertuples():
            if log.hopcount >= minHop and log.hopcount <= maxHop:
                draw(log,count,img_area,'lsgo',text)
        plt.imshow(img_area)

sigo: [6, 16, 69, 128, 133, 148, 257, 273, 301, 377] PDR: 1.0
lsgo: [6, 16, 69, 148, 257, 301, 377] PDR: 0.7
jbr: [6, 16, 69, 133, 148, 257, 273, 377] PDR: 0.8
sigo delay: 0.261522
lsgo delay: 0.29981
jbr delay: 0.3223375


interactive(children=(Dropdown(description='protocol', options=('both', 'lsgo', 'sigo', 'jbr'), value='both'),…

210 344

In [39]:
def output_df(hopcount, des_id):
    print('lsgo')
    display(lsgo_df[(lsgo_df['hopcount'] == hopcount) & (lsgo_df['destination_id'] == des_id)])
    print('sigo')
    display(sigo_df[(sigo_df['hopcount'] == hopcount) & (sigo_df['destination_id'] == des_id)])

In [314]:
@interact(hopcount=(0,21,1), des_id = dest_list)
def df_run(hopcount, des_id):
    output_df(hopcount, des_id)

interactive(children=(IntSlider(value=10, description='hopcount', max=21), Dropdown(description='des_id', opti…